In [1]:
import os
import glob
import pandas as pd
import numpy as np
os.chdir('./output')

In [2]:
path = './'
extension = '.parquet'
os.chdir(path)
batch = glob.glob('level2*{}'.format(extension))
batch

['level2_P103558.parquet',
 'level2_P103563.parquet',
 'level2_P103565.parquet',
 'level2_P103566.parquet',
 'level2_P103567.parquet',
 'level2_P103569.parquet',
 'level2_P103556.parquet',
 'level2_P103587.parquet',
 'level2_P103588.parquet',
 'level2_P103559.parquet',
 'level2_P103562.parquet',
 'level2_P103572.parquet',
 'level2_P103561.parquet',
 'level2_P103570.parquet',
 'level2_P103557.parquet',
 'level2_P103571.parquet',
 'level2_P103555.parquet',
 'level2_P103573.parquet',
 'level2_P103564.parquet',
 'level2_P103568.parquet',
 'level2_P103560.parquet']

In [3]:
#read metadata
meta = pd.read_csv('../metadata/LABDESIGN_metadata_SSS-A549-v2.csv', low_memory=False, sep=",") # change for each exp
meta = meta.rename(columns={
                     "barcode": "Metadata_plate_map_name",
                     "batch_id": "Metadata_cmpdName",
                     "well_id": "Metadata_Well",
                     "cell_line": "Metadata_CellLine",
                     "cmpd_conc": "Metadata_cmpdConc",
                     "cells_per_well": "Metadata_CellDensity"},
            errors="raise")

meta = meta[['Metadata_plate_map_name','Metadata_cmpdName','Metadata_cmpdConc','Metadata_Well']]
meta.tail(5)

,Metadata_plate_map_name,Metadata_cmpdName,Metadata_cmpdConc,Metadata_Well
8343,P103588,DMSO,0.3,P19
8344,P103588,DMSO,0.3,P20
8345,P103588,DMSO,0.3,P21
8346,P103588,DMSO,0.3,P22
8347,P103588,DMSO,0.3,P23


In [6]:
collected_df = []
for plate in batch:
    df = pd.read_parquet(plate)
    initial_shape = df.shape[0]  
    
    #print(f"{plate}: Initial number of rows = {initial_shape}")
    df = pd.merge(left=meta, right=df, left_on=['Metadata_plate_map_name','Metadata_Well'], right_on=['Metadata_plate_map_name','Metadata_Well'])
    final_shape = df.shape[0] 
    print(f"{plate}: Final number of rows after merging = {final_shape}")
    if initial_shape == final_shape:
        print(f"{plate}: All rows successfully matched with metadata.")
    else:
        print(f"{plate}: WARNING! {initial_shape - final_shape} rows were not matched with metadata.")
    
    df.to_parquet(f"level4_{plate.split('_')[1]}")
    collected_df.append(df)

level2_P103558.parquet: Final number of rows after merging = 3386
level2_P103558.parquet: WARNING! 9 rows were not matched with metadata.
level2_P103563.parquet: Final number of rows after merging = 3381
level2_P103563.parquet: WARNING! 9 rows were not matched with metadata.
level2_P103565.parquet: Final number of rows after merging = 3359
level2_P103565.parquet: All rows successfully matched with metadata.
level2_P103566.parquet: Final number of rows after merging = 3391
level2_P103566.parquet: WARNING! 9 rows were not matched with metadata.
level2_P103567.parquet: Final number of rows after merging = 3397
level2_P103567.parquet: All rows successfully matched with metadata.
level2_P103569.parquet: Final number of rows after merging = 3385
level2_P103569.parquet: WARNING! 9 rows were not matched with metadata.
level2_P103556.parquet: Final number of rows after merging = 3406
level2_P103556.parquet: WARNING! 9 rows were not matched with metadata.
level2_P103587.parquet: Final number of 

In [10]:
# merge multiple plates
dfs = pd.concat(collected_df)
dfs = dfs.loc[:, ~dfs.columns.str.contains(r'Unnamed|flag|_ObjectNumber', case=False, regex=True)]
dfs.to_parquet("level4_All.parquet", index=False)